In [1]:
from scraper import get_current_events, get_description_and_more_data, get_headers_and_cookies
from classes import Event, DB, Filter

40


In [13]:
data = get_description_and_more_data("Explore - 3D")
print(data)

https://www.planetarium-hamburg.de/files/default/img/hamburg--logo.png
{'description': '\n', 'img_url': 'https://www.planetarium-hamburg.de/files/default/img/hamburg--logo.png'}
